In [2]:
# from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import pandas as pd

hunt = pd.read_csv('Data\\Hunt\\huntfile.csv')
khar = pd.read_csv('Data\\Kharchenko\\kharchenkofile.csv')
xmatch = pd.read_csv('Data\\Hunt\\xmatchfile.csv')

"""In case you want to use fits files"""
# def datahandler(filename):
#     authorname = filename.split('\\')
#     hdul = fits.open(filename)
#     hdr1 = hdul[0].header #first and second header
#     hdr2 = hdul[1].header
#     data, cols = hdul[1].data, hdul[1].columns
#     metadata = [ [ hdr2[f"TTYPE{i+1}"], hdr2.comments[f"TTYPE{i+1}"] ] for i, j in enumerate(cols.names) ]
#     # mydict[authorname]['hdul'] = hdul
#     return hdul[1].data, hdul[1].columns, metadata

xmatch

,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,pmDESep,pmDESigma,PlxSep,PlxSigma,maxSigma,meanSigma
0,0,1636-283,Kharchenko+13,hipparcos,0.000298,0.002366,0.003970,0.002366,3.4450,0.662721,1.69600,0.3213,0.04456,0.0,0.663,0.3280
1,0,1636-283,Kharchenko+13,position,0.000298,0.002366,0.003970,0.002366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2MASS-GC01,Bica+18,position,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2MASS-GC02,Bica+18,position,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2MASS-GC02,Kharchenko+13,"position,hipparcos",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29940,7162,vdBergh_85,Kharchenko+13,position,0.009891,0.024375,0.076088,0.024375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29941,7162,vdBergh_85,Dias+02,position,0.010300,0.025379,0.308965,0.025379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29942,7163,vdBergh_92,Kharchenko+13,position,0.089140,0.185433,0.356566,0.185433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29943,7163,vdBergh_92,Cantat-Gaudin+20,gaia_dr2,0.032700,0.068013,0.145961,0.068013,0.1545,0.527101,0.03532,0.0000,0.03385,0.0,0.527,0.1757


### Getting Kharchenko numbers
Kharchenko clusters: $n_\mathrm{clusters}=2935 $, $n_\mathrm{detected}= 1513 \newline$ 
Steps:
1. Get Kharchenko from crossmatch table
2. Remove records with Sep (seperation between cluster centres) == NaN and remove duplicate ID's
3. Check if crossmatched kharchenko 
4. 

In [27]:
kharchenko_xm = xmatch.query('SourceCat == "Kharchenko+13"').drop_duplicates(subset=['ID']).dropna(subset=['Sep']) #Get crossmatch table with seperation=NaN removed

#sort out the globular clusters in the hunt and kharchenko file
hunt_oc = hunt.query('Type=="o"')
khar_nonglob = khar.query('Type!="g"')

#Match xmatched kharchenko data to confirmed OC's in Hunt catalogue
kharhunt = pd.merge(kharchenko_xm, hunt_oc, on= 'ID', how='inner')
khar
#See how many of the confirmed OC's from Hunt occur in the literature (Kharchenko) catalogue
# kharmerge = pd.merge(kharhunt, khar_nonglob, left_on= 'NameLit', right_on='Name', how='inner' )

#Same step as above, but now checking through all names from Hunt Catalogue
khar_expanded = kharhunt.assign(synonym=kharhunt['AllNames'].str.split(',')).explode('synonym')
allnamemerge = pd.merge(khar_expanded, khar_nonglob, left_on='synonym', right_on='Name', how='inner' )
allnamemerge.sort_values('ID')
allnamemerge

,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,...,e_rc,rt_y,e_rt,k,e_k,Src,SType,[Fe/H],e_[Fe/H],o_[Fe/H]
0,114,ASCC_10,Kharchenko+13,hipparcos,0.164400,0.031312,0.310144,0.031312,1.0470,0.274158,...,0.69,8.95,4.21,2.55,0.55,COCD,NaN,99.999,9.99,0
1,50,ASCC_100,Kharchenko+13,position,0.025990,0.008916,0.071212,0.008916,NaN,NaN,...,0.28,3.25,0.49,86.95,24.65,COCD,NaN,99.999,9.99,0
2,51,ASCC_101,Kharchenko+13,position,0.025590,0.008428,0.051181,0.008428,NaN,NaN,...,0.55,4.84,2.31,6.11,1.78,COCD,NaN,99.999,9.99,0
3,5400,ASCC_103,Kharchenko+13,position,0.048860,0.017086,0.174505,0.017086,NaN,NaN,...,0.26,6.90,3.25,8.13,2.60,COCD,NaN,99.999,9.99,0
4,52,ASCC_104,Kharchenko+13,position,0.063640,0.043635,0.159102,0.043635,NaN,NaN,...,1.51,7.74,1.10,15.43,10.04,COCD,NaN,99.999,9.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,7165,vdBergh_130,Kharchenko+13,position,0.073390,0.307565,0.638192,0.307565,NaN,NaN,...,0.27,5.31,2.05,3.87,1.31,DIAS,NaN,99.999,9.99,0
1509,7160,vdBergh_80,Kharchenko+13,position,0.028140,0.026219,0.216444,0.026219,NaN,NaN,...,0.21,8.79,3.70,6.41,2.44,DIAS,NaN,99.999,9.99,0
1510,7161,vdBergh_83,Kharchenko+13,position,0.114700,0.154470,0.955673,0.154470,NaN,NaN,...,0.17,7.34,4.74,8.88,5.93,DIAS,NaN,99.999,9.99,0
1511,7162,vdBergh_85,Kharchenko+13,hipparcos,0.009891,0.024375,0.076088,0.024375,0.2275,0.043188,...,0.25,5.00,0.86,6.28,1.22,DIAS,NaN,99.999,9.99,0
